## 파일에 있는 모든 이미지의 배경 제거

In [ ]:
from rembg import remove
import cv2
import numpy as np
from PIL import Image
import io
import os

# 입력 이미지 폴더 및 결과 이미지 폴더 경로
input_folder_path = 'sqt/sqt_0301/squat_1_image'
output_folder_path = 'sqt/0305_NoBG/1'

# 결과 이미지 폴더가 없으면 생성
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# 입력 폴더 내의 모든 이미지 파일에 대해 반복
for image_file in os.listdir(input_folder_path):
    # 파일 확장자 확인 (.jpg, .png 등 이미지 파일인 경우에만 진행)
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg')):
        input_image_path = os.path.join(input_folder_path, image_file)
        output_image_path = os.path.join(output_folder_path, image_file)
        
        # 이미지를 읽고 배경을 제거합니다.
        with open(input_image_path, 'rb') as i:
            input_image = i.read()
            output_image = remove(input_image)
        
        # 결과 이미지를 저장합니다.
        with open(output_image_path, 'wb') as o:
            o.write(output_image)

print("Background removal completed for all images.")


## 이미지 파일 이름 재설정
### 모델을 실행할 때 utf-8 문제 해결을 위함

In [ ]:
import os

# 파일이 위치한 디렉토리
directory = 'squat_240222_2100/squat_1_image'

# 파일 이름 변경을 위한 카운터
file_number = 1

# 디렉토리 내의 모든 파일에 대해 반복
for filename in os.listdir(directory):
    # 파일의 전체 경로 구성
    old_file = os.path.join(directory, filename)
    
    # 새 파일명 생성
    new_file = os.path.join(directory, f"1_{file_number}.jpg")
    
    # 파일 이름 변경
    os.rename(old_file, new_file)
    
    # 파일 번호 증가
    file_number += 1

print("모든 파일이 변경되었습니다.")

### frame 단위로 찍힌 이미지가 저장된 폴더 경로까지 설정하여 그 폴더에 이미지가 바로바로 저장되도록 저장. 20초가 지나면 꺼짐.

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

webcam = cv2.VideoCapture(0)
save_path = "saved_images1"
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
image_num = 0

start_time = time.time()

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while webcam.isOpened():
        ret, frame = webcam.read()
        
        if not ret :
            print("Ignoring empty camera frame.")
            continue
        
        elapsed_time = time.time() - start_time
        if elapsed_time > 20:
            break
        
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
            
        cv2.imshow('Mediapipe Feed', image)
        cv2.imwrite(os.path.join(save_path, f'image_{image_num:04d}.jpg'), image)
        image_num += 1
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    webcam.release()
    cv2.destroyAllWindows()